In [1]:
# Connect to the Cassandra cluster
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB, 
# if yours is different update the file name below
cloud_config = {
    'secure_connect_bundle': 'secure-connect-eduflow-ai-hack-2023.zip'
}

# This token JSON file is autogenerated when you download your token, 
# if yours is different update the file name below
with open("xavier@everdawn.ai-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.11-fedbfc208e4e


In [4]:
# Choose the keyspace
session.set_keyspace('aihack2023')

# Check the current keyspace
print("Current keyspace: ", session.keyspace)

Current keyspace:  aihack2023


# Create the tables
The tables that I need are as follows:
- Student table
    - School
    - Student ID No
    - First Name
    - Last Name
    - Email Address (PK)
    - Phone Number
    - Interests <>
    - Career Goals <>
    - Subjects <>

- Subject Table
    - Subject code (PK)
    - Subject Name

- Skill
    - Skill Code (PK)
    - Subject code
    - Skill Title
    - Skill Theory



In [6]:
# Students Table
cql_command = """
DROP TABLE IF EXISTS students;
"""

session.execute(cql_command)
print("Students table successfully dropped")

cql_command = """
CREATE TABLE students (
    email_address text PRIMARY KEY,
    school text,
    student_id_no text,
    first_name text,
    last_name text,
    phone_number text,
    interests list<text>,
    career_paths list<text>,
    subjects list<text>
    );
"""

session.execute(cql_command)
print("Students table successfully created")

Students table successfully dropped
Students table successfully created


In [8]:
# Subjects Table
cql_command = """
DROP TABLE IF EXISTS subjects;
"""

session.execute(cql_command)
print("Subjects table successfully dropped")

cql_command = """
CREATE TABLE subjects (
    subject_code text PRIMARY KEY,
    subject_name text,
    skills map<text, text>
);
"""

session.execute(cql_command)
print("Subjects table successfully created")

Subjects table successfully dropped
Subjects table successfully created


In [9]:
# Student_Skills Table
cql_command = """
DROP TABLE IF EXISTS student_skills;
"""

session.execute(cql_command)
print("student_skills table successfully dropped")

cql_command = """
CREATE TABLE student_skills (
    student_email text,
    subject_code text,
    skill_title text,
    mastery_score float,
    retention_score float,
    need_to_revise boolean,
    PRIMARY KEY ((student_email, subject_code), skill_title)
);
"""

session.execute(cql_command)
print("Student_skills table successfully created")

student_skills table successfully dropped
Student_skills table successfully created


In [11]:
def create_new_student(session, 
                       email_address, 
                       school, 
                       student_id_no, 
                       first_name, 
                       last_name, 
                       phone_number, 
                       interests, 
                       career_paths, 
                       subjects):
    """
    Creates a new student in the database, populating the students_skills table with all of the skills if they have subjects

    Args:
        session (database session): the database session
        email_address (string): the email address of the student
        school (string): the school of the student
        student_id_no (string): the student's ID number
        first_name (string): the student's first name
        last_name (string): the student's last name
        phone_number (string): the student's phone number
        interests (list): the student's interests
        career_paths (list): the student's career paths
        subjects (list): the student's subjects

    Returns:
        None

    Raises:
        Description of any errors that are raised.
    """
    cql_command = """
    INSERT INTO students (email_address, school, student_id_no, first_name, last_name, phone_number, interests, career_paths, subjects)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """

    session.execute(cql_command, [email_address, school, student_id_no, first_name, last_name, phone_number, interests, career_paths, subjects])
    print("New student successfully created")

In [12]:
create_new_student(session,
                   "xand0001@student.monash.edu",
                   "Monash University",
                   "28748115",
                   "Xavier",
                   "Andueza",
                   "0456488353",
                   ["Hiking","Gaming","Politics"],
                   ["Software Engineering","Data Science"],
                   ["FIT3171"])

New student successfully created


In [14]:
cql_command = """
INSERT INTO subjects (subject_code, subject_name, skills)
VALUES (%s, %s, %s);
"""

session.execute(cql_command, ["FIT3171", "DataBases", {"Can Create a Database": "Theory on databases",
                                                       "Can Insert Data into a database":"theory"}])

InvalidRequest: Error from server: code=2200 [Invalid query] message="Unexpected receiver type 'map<text, text>'; only list and vector are expected"

In [38]:
cql_command = """
            SELECT skills FROM subjects
            WHERE subject_code = %s;
            """

answer = session.execute(cql_command, ["FIT3171"])

In [42]:
print(answer.one().skills)

{'Can Create a Database': 'Theory on databases', 'Can Insert Data into a database': 'theory'}


In [43]:
skills_dict = answer.one().skills

In [50]:
print(list(skills_dict.keys()))

['Can Create a Database', 'Can Insert Data into a database']


In [52]:
cql_command = """
INSERT INTO student_skills (student_email, 
    subject_code, 
    skill_title, 
    mastery_score, 
    retention_score, 
    need_to_revise)
    VALUES (%s, %s, %s, %s, %s, %s);
"""

for skill in list(skills_dict.keys()):
    print(skill)

    session.execute(cql_command, ["xand0001@student.monash.edu",
                                  "FIT3171",
                                  skill,
                                  0.0,
                                  0.0,
                                  False])


Can Create a Database
Can Insert Data into a database


In [49]:
skills_dict[0]

AttributeError: 'int' object has no attribute 'encode'

In [31]:
print(answer[0])

{'Can Create a Database': 'Theory on databases', 'Can Insert Data into a database': 'theory'}


In [32]:
for skill in answer[0]:
    print(skill)

Can Create a Database
Can Insert Data into a database
